<a href="https://colab.research.google.com/github/malakkkamrr/Natural-Language-Processing-Project/blob/main/Nlp_milestone_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import sklearn.metrics

import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df_train = pd.read_csv("/content/final_dataset (2) (2).csv")
df_train.head()

,title,brand,description,price/currency,price/value,stars,reviewsCount
0,"65W Universal USB C Chromebook Charger,Replace...",NELEBUTO,"65W Universal USB C Chromebook Charger,Replace...",$,13.69,4.20,55
1,"EMPSIGN Laptop Tote bag for women 16 Inch, Sty...",EMPSIGN,"EMPSIGN Laptop Tote bag for women 16 Inch, Sty...",$,31.99,4.90,58
2,"100Pcs Mario Stickers, Cartoon Game Mario Stic...",MOQIAN,100Pcs Japanese Cartoon Game Mari Stickers for...,$,5.99,4.60,42
3,"USB Docking Station, JESWO USB 3.0 Laptop Dock...",JESWO,"USB Docking Station, JESWO USB 3.0 Laptop Dock...",$,79.99,4.20,39
4,ANCBD C31N1824 Battery for Asus Chromebook Fli...,ANCBD,Specification: Battery Rating: 11.55V Battery ...,$,46.99,4.53,942


In [ ]:
df_train = df_train.drop(['brand', 'description', 'price/currency', 'stars', 'reviewsCount' ], axis=1)
df_train.head()

,title,price/value
0,"65W Universal USB C Chromebook Charger,Replace...",13.69
1,"EMPSIGN Laptop Tote bag for women 16 Inch, Sty...",31.99
2,"100Pcs Mario Stickers, Cartoon Game Mario Stic...",5.99
3,"USB Docking Station, JESWO USB 3.0 Laptop Dock...",79.99
4,ANCBD C31N1824 Battery for Asus Chromebook Fli...,46.99


In [ ]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)


train_df.head()

,title,price/value
249,MOYYI Vintage Leather Laptop Backpack for Wome...,29.99
433,"DESEACO Aesthetic Watercolor Dreamy Feather, B...",7.99
19,Lacdo 360° Protective Laptop Sleeve Case for 1...,20.99
322,MOSISO Laptop Sleeve Compatible with MacBook A...,15.99
332,YINOVEEN Tempered Glass Screen Protector For 2...,17.99


In [ ]:
val_df.head()

,title,price/value
361,50 Packs Hold Flowers Stickers Water Bottles L...,5.950000
73,"50PCS Singer Stickers, Vinyl Waterproof Sticke...",5.990000
374,HP SA04XL 15.12V 55.67Wh Laptop Battery For HP...,49.990000
155,"pcnearty 100 Pcs Basketball Stickers for Kids,...",7.990000
104,200Pcs Inspirational Christian Stickers for Wa...,38.341702


In [ ]:
val_df.count()

title          100
price/value    100
dtype: int64

In [ ]:
train_df.count()

title          400
price/value    400
dtype: int64

In [ ]:
def process_text_column(df, column_name):
    df[column_name] = df[column_name].apply(remove_stopwords_and_punctuation_from_text)
    return df

def remove_stopwords_and_punctuation_from_text(text):
    nltk_stop_words = set(stopwords.words('english'))

    tokens = nltk.word_tokenize(text)

    punctuation ='!"$%&()*,-./:;<=>?@[\\]^_`{|}~'
    filtered_tokens = [word for word in tokens if word.lower() not in nltk_stop_words and word not in punctuation]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
train_df = process_text_column(train_df, 'title')
train_df.head()

,title,price/value
249,MOYYI Vintage Leather Laptop Backpack Women Me...,29.99
433,DESEACO Aesthetic Watercolor Dreamy Feather Bu...,7.99
19,Lacdo 360° Protective Laptop Sleeve Case 15 in...,20.99
322,MOSISO Laptop Sleeve Compatible MacBook Air/Pr...,15.99
332,YINOVEEN Tempered Glass Screen Protector 2023 ...,17.99


In [ ]:
val_df = process_text_column(val_df, 'title')
val_df.head()

,title,price/value
361,50 Packs Hold Flowers Stickers Water Bottles L...,5.950000
73,50PCS Singer Stickers Vinyl Waterproof Sticker...,5.990000
374,HP SA04XL 15.12V 55.67Wh Laptop Battery HP Env...,49.990000
155,pcnearty 100 Pcs Basketball Stickers Kids Wate...,7.990000
104,200Pcs Inspirational Christian Stickers Water ...,38.341702


In [ ]:
train_df['price/value'] = train_df['price/value'].round(2)
train_df.head()

,title,price/value
249,MOYYI Vintage Leather Laptop Backpack Women Me...,29.99
433,DESEACO Aesthetic Watercolor Dreamy Feather Bu...,7.99
19,Lacdo 360° Protective Laptop Sleeve Case 15 in...,20.99
322,MOSISO Laptop Sleeve Compatible MacBook Air/Pr...,15.99
332,YINOVEEN Tempered Glass Screen Protector 2023 ...,17.99


In [ ]:
val_df['price/value'] = val_df['price/value'].round(2)
val_df.head()

,title,price/value
361,50 Packs Hold Flowers Stickers Water Bottles L...,5.95
73,50PCS Singer Stickers Vinyl Waterproof Sticker...,5.99
374,HP SA04XL 15.12V 55.67Wh Laptop Battery HP Env...,49.99
155,pcnearty 100 Pcs Basketball Stickers Kids Wate...,7.99
104,200Pcs Inspirational Christian Stickers Water ...,38.34


In [ ]:
train_df['text_count']=train_df['title'].apply(nltk.word_tokenize).apply(len)
train_df.head()

,title,price/value,text_count
249,MOYYI Vintage Leather Laptop Backpack Women Me...,29.99,13
433,DESEACO Aesthetic Watercolor Dreamy Feather Bu...,7.99,22
19,Lacdo 360° Protective Laptop Sleeve Case 15 in...,20.99,30
322,MOSISO Laptop Sleeve Compatible MacBook Air/Pr...,15.99,26
332,YINOVEEN Tempered Glass Screen Protector 2023 ...,17.99,30


In [ ]:
train_df['text_count'].max()

46

In [ ]:
val_df['text_count']=val_df['title'].apply(nltk.word_tokenize).apply(len)
val_df.head()

,title,price/value,text_count
361,50 Packs Hold Flowers Stickers Water Bottles L...,5.95,25
73,50PCS Singer Stickers Vinyl Waterproof Sticker...,5.99,20
374,HP SA04XL 15.12V 55.67Wh Laptop Battery HP Env...,49.99,23
155,pcnearty 100 Pcs Basketball Stickers Kids Wate...,7.99,24
104,200Pcs Inspirational Christian Stickers Water ...,38.34,22


In [ ]:
val_df['text_count'].max()

40

In [ ]:
X_train = train_df['title']
Y_train = train_df['price/value'].values

In [ ]:
X_train[:3]

249    MOYYI Vintage Leather Laptop Backpack Women Me...
433    DESEACO Aesthetic Watercolor Dreamy Feather Bu...
19     Lacdo 360° Protective Laptop Sleeve Case 15 in...
Name: title, dtype: object

In [ ]:
Y_train[:3]

array([29.99,  7.99, 20.99])

In [ ]:
X_val = val_df['title']
Y_val = val_df['price/value'].values

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
embedding_dim = 64
units = 128
trunc_type='post'
padding_type='post'
max_length=50

In [ ]:
tokenizer = Tokenizer(num_words=None,
                      filters='!"$%&()*,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False,
                      oov_token=None)



In [ ]:
tokenizer.fit_on_texts(df_train['title'].values)

In [ ]:
tokenizer.word_index

{'for': 1,
 'laptop': 2,
 'stickers': 3,
 'with': 4,
 '15': 5,
 'backpack': 6,
 'inch': 7,
 'bag': 8,
 'usb': 9,
 'case': 10,
 'water': 11,
 'waterproof': 12,
 'macbook': 13,
 '3': 14,
 'c': 15,
 'women': 16,
 'pro': 17,
 'compatible': 18,
 'computer': 19,
 'vinyl': 20,
 'travel': 21,
 '6': 22,
 'and': 23,
 '2': 24,
 '13': 25,
 'kids': 26,
 'black': 27,
 'hp': 28,
 'charger': 29,
 'phone': 30,
 'screen': 31,
 'dell': 32,
 'air': 33,
 'bottle': 34,
 'adapter': 35,
 'men': 36,
 'lenovo': 37,
 'sticker': 38,
 'notebook': 39,
 'teens': 40,
 'bottles': 41,
 'cover': 42,
 '17': 43,
 'skateboard': 44,
 '1': 45,
 'm2': 46,
 '14': 47,
 'replacement': 48,
 '16': 49,
 'hard': 50,
 'adults': 51,
 'luggage': 52,
 'power': 53,
 'battery': 54,
 'sleeve': 55,
 'series': 56,
 'cute': 57,
 'station': 58,
 'girls': 59,
 'protector': 60,
 'work': 61,
 'docking': 62,
 '2023': 63,
 'college': 64,
 '50pcs': 65,
 'decals': 66,
 'stand': 67,
 'm1': 68,
 'in': 69,
 '4': 70,
 'pack': 71,
 'anti': 72,
 'keyboard'

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
X_train = tokenizer.texts_to_sequences(train_df['title'].values)
X_test = tokenizer.texts_to_sequences(val_df['title'].values)

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

In [ ]:
X_train[:3]

array([[ 793,  196,  108,    2,    6,   16,   36,   64,    6,  112,    5,
         848,    2,   27,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [2470,  131, 2471, 2472, 2473,  869,  992, 2474,  357,    3,  924,
        2475,   39,  107,   30,   10, 2476, 2477, 2478,  570,  993, 2479,
        2480,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [1074,  682,   94,    2,   55,   10,    5,    7,  124,   13,   33,
          46,   63,  168,    5,    7,   13,   17,  513,  514,   32,  214,
           5,  353,  114,  211,   19,    8,  683,   11,  143,   27,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   

In [ ]:
Y_train = train_df['price/value'].values
Y_test = val_df['price/value'].values

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
# ])

# # Set the training parameters for regression
# model.compile(
#     optimizer='adam',
#     loss='mean_squared_error',  # Appropriate loss for regression
#     metrics=['mean_absolute_error']
#     )  # Appropriate metric for regression

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(units, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(units)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mean_absolute_error']
)

In [ ]:
model.fit(X_train, Y_train, epochs=30)

model.evaluate(X_test, Y_test)

Epoch 1/30
13/13 [==============================] - 14s 308ms/step - loss: 35.0912 - mean_absolute_error: 35.0912
Epoch 2/30
13/13 [==============================] - 4s 290ms/step - loss: 26.4517 - mean_absolute_error: 26.4517
Epoch 3/30
13/13 [==============================] - 6s 453ms/step - loss: 25.7984 - mean_absolute_error: 25.7984
Epoch 4/30
13/13 [==============================] - 4s 301ms/step - loss: 25.8202 - mean_absolute_error: 25.8202
Epoch 5/30
13/13 [==============================] - 4s 289ms/step - loss: 25.8123 - mean_absolute_error: 25.8123
Epoch 6/30
13/13 [==============================] - 6s 457ms/step - loss: 25.3360 - mean_absolute_error: 25.3360
Epoch 7/30
13/13 [==============================] - 4s 291ms/step - loss: 23.8229 - mean_absolute_error: 23.8229
Epoch 8/30
13/13 [==============================] - 4s 300ms/step - loss: 20.7610 - mean_absolute_error: 20.7610
Epoch 9/30
13/13 [==============================] - 6s 438ms/step - loss: 20.3919 - mean_absolu

[19.625139236450195, 19.625139236450195]

In [ ]:
testing_text=val_df['title'][73]
actual_class=val_df['price/value'][73]

In [ ]:
actual_class

5.99

In [ ]:
test_seq=tokenizer.texts_to_sequences([testing_text])
test_seq=pad_sequences(test_seq, padding='post', maxlen=max_length)
predictions=model.predict(test_seq)
predictions

1/1 [==============================] - 2s 2s/step


array([[5.2853813]], dtype=float32)